In [29]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse

In [30]:
def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

In [31]:
df_path = "./predictions/tobii_test_more_layers_weighted_loss_more_patience_more_complex_tune.csv"
df_path_golden = "./golden_test.csv"
user_specs_path = "../data/participant_characteristics.csv"
user_specs = pd.read_csv(user_specs_path)[[
    "Display Height (pixels)", "Display Width (pixels)", 'Participant ID',
    'Screen Width (cm)', 'Screen Height (cm)', 'Distance From Screen (cm)',
    'Self-Reported Vision'
]]
df_pred = pd.read_csv(df_path)
df_pred["distances"] = df_pred[['x_normalized', 'y_normalized', 'pred_x', 'pred_y']].apply(lambda x: distance(*x), axis = 1)
df_pred["user_id"] = df_pred["full_name"].apply(lambda x: x.split("#")[0])
df_pred = df_pred.merge(user_specs, left_on = "user_id", right_on = 'Participant ID', how = "inner", suffixes=["", "_"]).drop(columns = ['Participant ID'])

df_golden = pd.read_csv(df_path_golden)
df_pred_golden = df_golden.merge(df_pred, on = "paths", how = "inner", suffixes=["", "_"])

In [32]:
df_pred.head()

,paths,x_normalized,y_normalized,full_name,pred_x,pred_y,distances,user_id,Display Height (pixels),Display Width (pixels),Screen Width (cm),Screen Height (cm),Distance From Screen (cm),Self-Reported Vision
0,/home/ubuntu/projects/tweakle/data/tobii_image...,0.194715,0.582539,P_38#1492618354027_23_-study-tooth_abscess_wri...,0.299071,0.539524,0.112874,P_38,900,1440,33.17,20.73,50.0,Normal
1,/home/ubuntu/projects/tweakle/data/tobii_image...,0.200532,0.493025,P_38#1492618354027_16_-study-educational_advan...,0.267034,0.464602,0.072321,P_38,900,1440,33.17,20.73,50.0,Normal
2,/home/ubuntu/projects/tweakle/data/tobii_image...,0.247674,0.453015,P_38#1492618354027_24_-study-dot_test_final_in...,0.234997,0.383104,0.071052,P_38,900,1440,33.17,20.73,50.0,Normal
3,/home/ubuntu/projects/tweakle/data/tobii_image...,0.217074,0.442437,P_38#1492618354027_7_-study-benefits_of_runnin...,0.280526,0.475514,0.071556,P_38,900,1440,33.17,20.73,50.0,Normal
4,/home/ubuntu/projects/tweakle/data/tobii_image...,0.123454,0.445198,P_38#1492618354027_9_-study-benefits_of_runnin...,0.180465,0.595299,0.160563,P_38,900,1440,33.17,20.73,50.0,Normal


In [33]:
df_pred["Self-Reported Vision"].value_counts()

Self-Reported Vision
Normal      8078
Glasses     4426
Contacts    1496
Name: count, dtype: int64

In [34]:
def average_angle(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Screen Width (cm)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Screen Width (cm)"]
    df_["y"] = df_["y_normalized"] * df_["Screen Height (cm)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Screen Height (cm)"]
    df_["center_y"] = df_["Screen Height (cm)"] * 0.5
    df_["center_x"] = df_["Screen Width (cm)"] * 0.5
    df_["distances_center_pred"] = df_[['center_x', 'center_y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    df_["distances_center_gt"] = df_[['center_x', 'center_y', 'x', 'y']].apply(lambda x: distance(*x), axis = 1)
    df_["tg_center_gt"] = df_["distances_center_gt"] / df_["Distance From Screen (cm)"]
    df_["tg_center_pred"] = df_["distances_center_pred"] / df_["Distance From Screen (cm)"]
    df_["angle_gt"] = np.arctan(df_["tg_center_gt"].values) * 180 / np.pi
    df_["angle_pred"] = np.arctan(df_["tg_center_pred"].values) * 180 / np.pi
    angles = (df_["angle_gt"] - df_["angle_pred"]).abs()
    return angles.mean(), angles.std()

In [35]:
def average_pixel_distance(df):
    df_ = df.copy()
    df_["x"] = df_["x_normalized"] * df_["Display Width (pixels)"]
    df_["pred_x_"] = df_["pred_x"] * df_["Display Width (pixels)"]
    df_["y"] = df_["y_normalized"] * df_["Display Height (pixels)"]
    df_["pred_y_"] = df_["pred_y"] * df_["Display Height (pixels)"]
    df_["distances"] = df_[['x', 'y', 'pred_x_', 'pred_y_']].apply(lambda x: distance(*x), axis = 1)
    return df_["distances"].mean(), df_["distances"].std()

In [36]:
def compute_metrics(df):
    preds = df[["pred_x", "pred_y"]].values
    labels = df[["x_normalized", "y_normalized"]].values
    
    mape_value = mape(labels, preds)
    print(f"Test MAPE: {round(mape_value, 4)}")
    
    rmse_value = mse(labels, preds, squared=True)
    print(f"Test RMSE: {round(rmse_value, 4)}")
    
    av_rel_dist = df["distances"].mean()
    print(f"Average relative distance: {round(av_rel_dist, 4)} +- {round(df['distances'].std(), 3)}")

    av_pixel_dist, std = average_pixel_distance(df)
    print(f"Average pixel distance: {round(av_pixel_dist, 1)} +- {round(std, 1)}")
    av_angle, std = average_angle(df)
    print(f"Average angle: {round(av_angle, 2)} +- {round(std, 2)}")

All test

In [37]:
compute_metrics(df_pred)

Test MAPE: 0.4011
Test RMSE: 0.0132
Average relative distance: 0.1235 +- 0.106
Average pixel distance: 175.6 +- 142.9
Average angle: 2.29 +- 2.15


Glasses

In [38]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] == "Glasses"]

In [39]:
compute_metrics(df_glasses)

Test MAPE: 0.4594
Test RMSE: 0.0147
Average relative distance: 0.1319 +- 0.11
Average pixel distance: 186.6 +- 146.5
Average angle: 2.36 +- 2.24


Normal

In [40]:
df_glasses = df_pred[df_pred['Self-Reported Vision'] != "Glasses"]

In [41]:
compute_metrics(df_glasses)

Test MAPE: 0.3741
Test RMSE: 0.0125
Average relative distance: 0.1196 +- 0.103
Average pixel distance: 170.5 +- 140.9
Average angle: 2.26 +- 2.11


Golden test

In [42]:
compute_metrics(df_pred_golden)

Test MAPE: 1.428
Test RMSE: 0.0199
Average relative distance: 0.1687 +- 0.123
Average pixel distance: 173.5 +- 102.5
Average angle: 2.07 +- 2.5
